# Eficiência computacional

Avaliação do tempo computacional do modelo original e das versões do modelo

### Pacotes

In [1]:
using CSV;
using DataFrames;
using JuMP;
using Ipopt;
using Plots;
using SpecialFunctions;

### Modelagem
**ÍNDICES:**

Ativos - i ∈ {1,..,M}. M=39

Dias   - t ∈ {1,...,N}. N=3717

perfis - j ∈ {1,...,L}. L=3

In [2]:
M=39
N=3717
L=3

3

**PARÂMETROS:**

Taxa de retorno diário dos ativos- Matriz N x M


Perfil de risco - Vetor de tamanho L


Nível de confiança-Vetor de tamanho L

Retorno mínimo aceitável pelo investidor - Número entre 0 e 1


In [3]:
#Retorno
tabelaRetorno=CSV.File("C://Users//Rafaela//Desktop//Documentos Gerais//Tese-Finanças//dados//retornos40_c.csv");
DataRetornos = DataFrame(tabelaRetorno);
retornos=zeros(N,M)
retornos.=DataRetornos[1:N,2:(M+1)]

#Perfil de risco
χ=[1/3 1/3 1/3]

#nível de confiança
α=[0.6 , 0.75,  0.9 ]

#retorno mínimo aceitável pelo investidor
δ=0.1

0.1

In [16]:
χ=[0.1 0.2 0.7]

1×3 Matrix{Float64}:
 0.1  0.2  0.7

### Modelo 0 - modelo original

In [4]:
modelT0 = Model(Ipopt.Optimizer)

    @variables(modelT0, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0 
        y[1:L] 
        z[1:N,1:L] ≥ 0
        end)
 
    @constraint(modelT0,sum(x[i] for i=1:M)== 1);
    @constraint(modelT0, u[1]==0 );
    @constraint(modelT0,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT0, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
    @constraint(modelT0,[t=1:N,j=1:L], u[t]-y[j] ≤ z[t,j]);
  
    @objective(modelT0, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(z[t,j] for t=1:N )) for j=1:L));
   

In [5]:
    t0=time()
    optimize!(modelT0) #otimizar
    Δt0=time()-t0; Δt0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   174913
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    14910
                     variables with only lower bounds:    14868
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equal

34.40400004386902

In [20]:
    pesoT0= value.(x); 
    VaRT0= value.(y)
    zT0= value.(z);
    uT0=value.(u);
    CVaRmistoT0=round(objective_value(modelT0);digits=3);
   CVaRmistoT0
    sum(transpose(pesoT0)*retornos[t,:] for t=1:N)

2.205979055590429

### Modelo 1 - versão 1

In [9]:
modelT1 = Model(Ipopt.Optimizer)

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    #função substituta f1--------------------------------
    η=2^(-14)        #parâmetro da função f1, menor melhor
   
    f1(a) = max(a,0)    

    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       # a função f1
    d2f1(a)= η/(2*(η+a^2)^(3/2))    # a derivada da função f1
    
    #-------------------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
   
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 

In [10]:
    t1=time()
    optimize!(modelT1) #otimizar
    Δt1=time()-t1;    Δt1

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   141460
Number of nonzeros in Lagrangian Hessian.............:    14871

Total number of variables............................:     3759
                     variables with only lower bounds:     3717
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     3717
        inequality constraints with only lower bounds:     3717
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

10.555999994277954

In [11]:
    pesoT1= value.(x); 
    VaRT1= value.(y); 
    uT1=value.(u);
    CVaRmistoT1=round(objective_value(modelT1);digits=3); CVaRmistoT1;
    pesoT1[21:39];
    VaRT1

3-element Vector{Float64}:
 0.052899053298307186
 0.07517565620548262
 0.11393497118872833

### Modelo 2 -Versão 2

In [12]:
modelT2 = Model(Ipopt.Optimizer)

    @variables(modelT2, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT2,sum(x[i] for i=1:M)== 1);
    @constraint(modelT2, u[1]==0 );
    @constraint(modelT2,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT2, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    # função substituta f2--------------------------------
    η = 0.03  #parâmetro da função f2, menor melhor

    f2(a) = max(a,0)
    
    df2(a)=   #a função f2
    if a ≤ - η
        result= 0.0
    elseif - η < a ≤ η
        result = a/(2*η) + 1/2 
    elseif  a > η
        result = 1.0
    end
    
    d2f2(a)=   # a derivada da função f2
    if a ≤ - η
        result= 0.0
    elseif - η < a ≤ η
        result = 1/(2*η) 
    elseif  a > η
        result = 0.0
    end
    #----------------

JuMP.register(modelT2, :f2, 1, f2, df2, d2f2)
 
@NLobjective(modelT2, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f2(u[t]-y[j]) for t=1:N )) for j=1:L)); 


In [13]:

t2=time()
optimize!(modelT2) #otimizar
Δt2=time()-t2; Δt2

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   141460
Number of nonzeros in Lagrangian Hessian.............:    14871

Total number of variables............................:     3759
                     variables with only lower bounds:     3717
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     3717
        inequality constraints with only lower bounds:     3717
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

5.417999982833862

In [14]:
pesoT2= value.(x); 
VaRT2= value.(y); 
uT2=value.(u);
CVaRmistoT2=round(objective_value(modelT2);digits=3); CVaRmistoT2;
pesoT2[21:39];
VaRT2

3-element Vector{Float64}:
 0.053858826459392266
 0.0763667364619099
 0.11587329280392004

### Modelo 3 - Versão 3

In [15]:
modelT3 = Model(Ipopt.Optimizer)

    @variables(modelT3, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT3,sum(x[i] for i=1:M)== 1);
    @constraint(modelT3, u[1]==0 );
    @constraint(modelT3,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT3, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
   # função substituta f3--------------------------------
    η = 0.021  #parâmetro da função f3, menor melhor

    f3(a) = max(a,0)
   
    df3(a)=   #a função f3
    if a ≤ - η
        result= 0.0
    elseif - η < a ≤ η
        result = (-a^3)/(4*η^3)+ (3*a)/(4*η)+ 1/2
    elseif  a > η
        result = 1.0
    end

    d2f3(a)=   # a  derivada da função f3
    if a ≤ - η
        result= 0.0
    elseif - η < a ≤ η
        result = (-3*a^2)/(4*η^3)+ 3/(4*η)
    elseif  a > η
        result = 0.0
    end
    #------------

JuMP.register(modelT3, :f3, 1, f3, df3, d2f3)
 
@NLobjective(modelT3, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f3(u[t]-y[j]) for t=1:N )) for j=1:L)); 


In [16]:
t3=time()
optimize!(modelT3) #otimizar
Δt3=time()-t3; Δt3

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   141460
Number of nonzeros in Lagrangian Hessian.............:    14871

Total number of variables............................:     3759
                     variables with only lower bounds:     3717
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     3717
        inequality constraints with only lower bounds:     3717
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

4.7779998779296875

In [17]:
pesoT3= value.(x); 
VaRT3= value.(y);  
uT3 =value.(u); 
CVaRmistoT3=round(objective_value(modelT3);digits=3);CVaRmistoT3;
pesoT3[21:39];
VaRT3

3-element Vector{Float64}:
 0.05261729505523041
 0.07470040067729379
 0.11261742513752347

### Modelo 4- Versão 4

In [18]:
modelT4 = Model(Ipopt.Optimizer)

    @variables(modelT4, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT4,sum(x[i] for i=1:M)== 1);
    @constraint(modelT4, u[1]==0 );
    @constraint(modelT4,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT4, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    # função substituta f4--------------------------------
    β=4
    η = 0.00000027
    f4(a) = max(a,0)
       
    df4(a)=1/2*(a^(β-1)*(a^β+ η)^(1/β-1)+1) #a função f4

    d2f4(a)=1/2*((β-1)*η*a^(β-2)*(a^β+η)^(1/β-2))# a derivada da função f4
   
    #--------------------------------------
JuMP.register(modelT4, :f4, 1, f4, df4, d2f4)
 
@NLobjective(modelT4, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f4(u[t]-y[j]) for t=1:N )) for j=1:L)); 


In [19]:
t4=time()
optimize!(modelT4) #otimizar
Δt4=time()-t4; Δt4

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   141460
Number of nonzeros in Lagrangian Hessian.............:    14871

Total number of variables............................:     3759
                     variables with only lower bounds:     3717
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     3717
        inequality constraints with only lower bounds:     3717
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

7.948000192642212

In [20]:
pesoT4= value.(x); 
VaRT4= value.(y);  
uT4 =value.(u);  
CVaRmistoT4=round(objective_value(modelT4);digits=3);CVaRmistoT4;
pesoT4[21:39];
VaRT4

3-element Vector{Float64}:
 0.05576174573620935
 0.07912154670035722
 0.12061630220803173

### Modelo 5 - Versão 5

In [12]:
modelT5 = Model(Ipopt.Optimizer)

    @variables(modelT5, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT5,sum(x[i] for i=1:M)== 1);
    @constraint(modelT5, u[1]==0 );
    @constraint(modelT5,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT5, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );

   # função substituta f9--------------------------------
      ρ=400 #parâmetro
     
      f5(a) = max(a,0)
           
      df5(a)= atan(ρ*a)/π+1/2
    
      d2f5(a)= (ρ/(1+ρ^2*a^2))/π
    #------------------------------

JuMP.register(modelT5, :f5, 1, f5,  df5,d2f5)
 
@NLobjective(modelT5, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f5(u[t]-y[j]) for t=1:N )) for j=1:L)); 



In [13]:
t5=time()
optimize!(modelT5) #otimizar
Δt5=time()-t5;Δt5

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:   141460
Number of nonzeros in Lagrangian Hessian.............:    14871

Total number of variables............................:     3759
                     variables with only lower bounds:     3717
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:     3717
        inequality constraints with only lower bounds:     3717
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

6.918999910354614

In [14]:
pesoT5= value.(x); 
VaRT5= value.(y); 
uT5 =value.(u);  
CVaRmistoT5=round(objective_value(modelT5);digits=3); CVaRmistoT5;
pesoT5[21:39];
CVaRmistoT5

0.128